# MXNet/Gluon Inference

In [18]:
import os
import sys
import numpy as np
import mxnet as mx
from mxnet import gluon, nd
from collections import namedtuple
from common.params_inf import *
from common.utils import *

In [19]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [20]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.3 |Anaconda custom (64-bit)| (default, Oct 13 2017, 12:02:49) 
[GCC 7.2.0]
Numpy:  1.13.3
GPU:  ['Tesla V100-SXM2-16GB']
CUDA Version 9.0.176
CuDNN Version  7.0.5


In [21]:
Batch = namedtuple('Batch', ['data'])

In [22]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCH_SIZE*BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [23]:
# Download Resnet weights
path='http://data.mxnet.io/models/imagenet/'
mx.test_utils.download(path+'resnet/50-layers/resnet-50-symbol.json')
mx.test_utils.download(path+'resnet/50-layers/resnet-50-0000.params')
print("Downloaded")

Downloaded


In [24]:
# Load model
sym, arg_params, aux_params = mx.model.load_checkpoint('resnet-50', 0)
# List the last 10 layers
all_layers = sym.get_internals()

In [25]:
print(all_layers[len(all_layers)-10:])

<Symbol group [bn1_moving_var, bn1, relu1, pool1, flatten0, fc1_weight, fc1_bias, fc1, softmax_label, softmax]>


In [26]:
# Get last layer
flatten_layer = all_layers['flatten0_output']

In [27]:
# We create the network
net = gluon.nn.SymbolBlock(outputs=flatten_layer, inputs=mx.sym.var('data'))
ctx = mx.gpu()

In [28]:
# We assign the weights
net_params = net.collect_params()
for param in arg_params:
    if param in net_params:
        net_params[param]._load_init(arg_params[param], ctx=ctx)
for param in aux_params:
    if param in net_params:
        net_params[param]._load_init(aux_params[param], ctx=ctx)

In [29]:
# We hybridize the network
net.hybridize()

In [30]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = nd.zeros((len(data), RESNET_FEATURES), dtype=np.float32)
    for idx, dta in yield_mb_X(data, batchsize):
        outputs = classifier(mx.nd.array(dta, ctx=ctx))
        out[idx*batchsize:(idx+1)*batchsize] = outputs
    nd.waitall()
    return out.asnumpy()

In [31]:
cold_start = predict_fn(net, fake_input_data_cf, BATCH_SIZE)

In [32]:
%%time
features = predict_fn(net, fake_input_data_cf, BATCH_SIZE)

CPU times: user 1.46 s, sys: 424 ms, total: 1.88 s
Wall time: 1.39 s


In [34]:
print("Images per second {}".format((BATCH_SIZE*BATCHES_GPU)/1.39))

Images per second 920.8633093525181
